# BERT part of speech tagger

In [1]:
import torch
# If GPU available
if torch.cuda.is_available():    
    device = torch.device("cuda")
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('We will use the GPU:', torch.cuda.get_device_name(0))
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: GeForce GTX 1060 3GB


In [2]:
from batchify import *
from create_vocab import *
from data_handling import *
from parser import *
from projectivize import *
from uas import *
from window_models import *
from taggers import *

## Import data

In [3]:
train_data = Dataset('data/en_gum-ud-train-projectivized.conllu')
dev_data = Dataset('data/en_gum-ud-dev-projectivized.conllu')
test_data = Dataset('data/en_gum-ud-test-projectivized.conllu')

## Create tagger

## Train tagger

In [4]:
import torch.optim as optim
import torch
import torch.nn.functional as F

def train_fixed_window(train_data, n_epochs=2, batch_size=100, lr=1e-3):
    vocab_words, vocab_tags = make_vocabs(train_data)
    tagger = FixedWindowTagger(vocab_words, vocab_tags, len(vocab_tags))
    tagger.model.to(device)
    
    optimizer = optim.Adam(tagger.model.parameters(), lr=lr)
    for i in range(n_epochs):
        total_loss = 0
        batch_nr = 0
        for x, y in training_examples_tagger(vocab_words, vocab_tags, train_data, tagger):
            x = x.to(device)
            y = y.to(device)
            batch_nr += 1
            
            optimizer.zero_grad()
            y_pred = tagger.model.forward(x)
            
            loss = F.cross_entropy(y_pred, y)
            loss.backward()
            total_loss += loss.item()
            optimizer.step()
            #if batch_nr % 500 == 1:
                #print(total_loss/batch_nr)
                #pass
        print(f"Epoch {i}, loss: {total_loss/batch_nr}")
    return tagger


In [5]:
tagger = train_fixed_window(train_data, n_epochs=7)

Epoch 0, loss: 1.1983525420770291
Epoch 1, loss: 0.35111965324617317
Epoch 2, loss: 0.17210579288393052
Epoch 3, loss: 0.1081733201909162
Epoch 4, loss: 0.079901198323345
Epoch 5, loss: 0.06374528698067844
Epoch 6, loss: 0.05491415657596814


In [6]:
accuracy(tagger, train_data)

0.979070901239727

In [ ]:
accuracy(tagger, dev_data)